In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# введите имя папки на гугл диске, в которую вы сохранили папку vision,
# содержащую .py файлы, а также папки datatasets и classifiers например,
# cv/assignments/assignment1/vision/
FOLDERNAME = None

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My Drive
%cp -r $FOLDERNAME ../../
%cd ../../
%cd vision/datasets/
!bash get_datasets.sh
%cd ../../

# Многоклассовая классификация с помощью метода опорных векторов (SVM)

*Выполните задания и сдайте их, сохранив полученные результаты, вместе со всем необходимым кодом, включая код за пределами этого ноутбука, например, код в папке `vision`*

В этой лаборатрной вы:
    
- реализуете векторную версию функции потерь для SVM
- реализуете векторную версию аналитического градиента для функции потерь
- проверите свою реализацию с помощью численного нахождения градиента
- используя проверочную выборку найдете скорость обучения и степерь регуляризации
- оптимизируете функцию потерь с помощью стохастического градиентного спуска
- визуализируете найденные матрицы выесов


In [ ]:
# Этот код нужно запустить для предватирельных настроек

import random
import numpy as np
from vision.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# Чтобы графики выводились в том же окне, а не новом
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер изображений
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## CIFAR-10 Data Loading and Preprocessing

In [ ]:
# Загрузка выборки CIFAR-10.
cifar10_dir = 'vision/datasets/cifar-10-batches-py'

# Для того, чтобы данные не дублировались при повторной загрузке (например, если
# вы запускаете эту ячейку, не обновив сессию), очистим переменные
try:
   del X_train, y_train
   del X_test, y_test
   print('Загруженны данны удален')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# Убедимся, что данные загружены и выведем размеры обучающих и тестовых данных
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Выведем несколько примеров из выборки
# Покажем несколько изображений каждого класса
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Разделим данные на обучающую, проверочную и тестовую выборки. Также мы
# выберем небольшую "разработческую" выборку - подмножество обучающих 
# данных, ее мы будем использовать, чтобы на этапе разработке ждать
# резльутатов меньше.
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

# Проверочная выборка - num_validation изображений из обучающей выборки
mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]

# Обучающая выборка - первые num_train изображений из оригинальной выборки.
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]

# "Разработческая" выборка.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

# Тестовая выборка - перввые num_test изображений из оригинальной тестовой
# выборки.
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Предобратка: преобразуем изображения в строки матрицы
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

# Выведем размерность полученных данных
print('Training data shape: ', X_train.shape)
print('Validation data shape: ', X_val.shape)
print('Test data shape: ', X_test.shape)
print('dev data shape: ', X_dev.shape)

In [ ]:
# Предобработка: вычтем среднее значение пикселей в изобрежении
# Для начала вычислим среднее значение на обучаюзщей выборке
mean_image = np.mean(X_train, axis=0)
print(mean_image[:10])
plt.figure(figsize=(4,4))
plt.imshow(mean_image.reshape((32,32,3)).astype('uint8')) # визуализация среднего
plt.show()

# Затем вычтем полученное средние из наших данных
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# Допишем еще одно измерение, которе упростит работу с коэффициентом b
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

## Классификатор SVM

Код для этого раздела необходимо дописать в `vision/classifiers/linear_svm.py`. 

Функция `svm_loss_naive` реализует вычисление функции потерь с использованием циклов.

In [ ]:
# Проверим работу реализации, которую мы вам предоставили:
from vision.classifiers.linear_svm import svm_loss_naive
import time

# сгенерируем матрицу из небольших чисел
W = np.random.randn(3073, 10) * 0.0001 

loss, grad = svm_loss_naive(W, X_dev, y_dev, 0.000005)
print('loss: %f' % (loss, ))

Полученный градинет (переменная `grad`) должен быть примерно равным 0. Вычислите и реализуйте градиент для функции потерь в функции `svm_loss_naive`.

Для того, чтобы убедиться в том, что градиент найден верно, численно посчитаем градиент данной функции в некоторой точке и сравним его с нашей реализацией.

In [ ]:
# Вычислите аналитически градиент и проверьте его с помощью градиента,
# посчитанного численно (уже реализовано)

# Вычислим градиент и значение функции потерь для матрицы W.
loss, grad = svm_loss_naive(W, X_dev, y_dev, 0.0)

# Численно посчитаем градиент для несколькх выбранных измерений и сравним с
# нашим градентом, полученным аналитически. Числа должны практичиски совпадать
# для всех измерений.
from vision.gradient_check import grad_check_sparse
f = lambda w: svm_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad)

# Еще раз проверим точность вычисления градиента, на этот раз задав коэффициент
# регуляризации
loss, grad = svm_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: svm_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad)

In [ ]:
# Реализуйте функцию svm_loss_vectorized; для начала вичислите только значение
# функции потерь
tic = time.time()
loss_naive, grad_naive = svm_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('Naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from vision.classifiers.linear_svm import svm_loss_vectorized
tic = time.time()
loss_vectorized, _ = svm_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('Vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# Значения функции потерь должны совпадать, но векторная реализаци должна
# работать значительно быстрее
print('difference: %f' % (loss_naive - loss_vectorized))

In [ ]:
# Завершите реализацию svm_loss_vectorized, вычислив градиент функции потерь с
# помощью векторных операций

# Градиенты, реализованные с помощью векторных операций и без них, должны
# совпадать, но векторная реализация должна быть значительно быстрее
tic = time.time()
_, grad_naive = svm_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('Naive loss and gradient: computed in %fs' % (toc - tic))

tic = time.time()
_, grad_vectorized = svm_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('Vectorized loss and gradient: computed in %fs' % (toc - tic))

difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('difference: %f' % difference)

### Стохастический градиентый спуск

Теперь, когда у нас есть функции вычисляющие значение функции потерь и градент, мы можем приступить к реализации стохастического градиентного спуска. Код для этой части задания необходимо писать в файле `vision/classifiers/linear_classifier.py`.

In [ ]:
# В linear_classifier.py, реализуйте стохастический градиентный спуск в функции
# LinearClassifier.train() и затем запустите его
from vision.classifiers import LinearSVM
svm = LinearSVM()
tic = time.time()
loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
                      num_iters=1500, verbose=True)
toc = time.time()
print('That took %fs' % (toc - tic))

In [ ]:
# Выведем график зависимости функции потерь от номера итерации
plt.plot(loss_hist)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

In [ ]:
# Реализуйте функцию LinearSVM.predict и измерьте точность грассификации на
# проверочной и тестовой выборках
y_train_pred = svm.predict(X_train)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
y_val_pred = svm.predict(X_val)
print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))

In [ ]:
# Используя проверочную выборку (не тестовую!) настройте гиперпараметры -
# регуляризацию и скорость обучения. Проведите эксперимент с разными диапазонами 
# значений регляризации и скорости обучения. Вы должны получить точность 
# классификации, равную примерно 0,39 на проверочной выборке.

# Примечание: во время обучения могут возникнуть сообщения, содержащие слова "runtime", "overflow", это нормально.

# results - это словарь, сопоставляющий кортежи вида (learning_rate, 
# regularization_strength) с кортежами вида (training_accuracy, 
# validation_accuracy). Точность - доля верно классифицированных изображений
results = {}
best_val = -1   # Наибольшее значение точности, которое вы смогли получить
best_svm = None # Переменная, в которой будет хранится классификатор, давший наилудший результат

################################################################################
# TODO:                                                                        #
# Напишите программу, которая выбирает наилучшие параметры на проверочной      #
# выборке. Для каждой комбинации гиперпараметров, обучите SVM на тренировочной #
# выборке, вычислите точность классификации на проверочной выборке, и          #
# сохраните эти результаты в results. Также сохраните наилучшее значение       #
# точности в                                                                   #
# best_val и классификатор, давший эту точность сохраните в best_svm.          #
#                                                                              #
# Подсказка: используйте  небольше количество итераций (num_iters) при         #
# разработке, чтобы ускорить процесс разработки. Когда убедитесь, что все      #
# работает, увеличте количество итераций.                                      #
################################################################################

# Это лишь примерные значения гиперпараметров. Настраивайте их на свое усмотрение
learning_rates = [1e-7, 5e-5]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# Визуализируем результат кросс-валидации
import math
import pdb

x_scatter = [math.log10(x[0]) for x in results]
y_scatter = [math.log10(x[1]) for x in results]

# Попробуйте разобраться, что показывают данные графики
marker_size = 100
colors = [results[x][0] for x in results]
plt.subplot(2, 1, 1)
plt.tight_layout(pad=3)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 training accuracy')

colors = [results[x][1] for x in results]
plt.subplot(2, 1, 2)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 validation accuracy')
plt.show()

In [ ]:
# Вычислим точность наилучшего классификатора на тестовой выборке
y_test_pred = best_svm.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('linear SVM on raw pixels final test set accuracy: %f' % test_accuracy)

In [ ]:
# Визуализируем веса, полученные в результате обучения для каждого класса
# В зависимости от того, на сколько аккуратно вы подобрали гиперпараметры,
# результат может получиться красивым или не очень
# Visualize the learned weights for each class
w = best_svm.W[:-1,:] # уберем дополнительное измерение, которое мы добавили для обучения
w = w.reshape(32, 32, 3, 10)
w_min, w_max = np.min(w), np.max(w)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
      
    # Веса должны быть в диапазоне от 0 до 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])